In [1]:
import argparse
import cv2
import numpy as np
import torch
from torch._C import _get_warnAlways
from torchvision import models
from torchvision import transforms
from torchvision.transforms import Compose, Normalize, ToTensor

def preprocess_image(img: np.ndarray, mean=None, std=None) -> torch.Tensor:
    if std is None:
        std = [0.5, 0.5, 0.5]
    if mean is None:
        mean = [0.5, 0.5, 0.5]

    preprocessing = Compose([
        ToTensor(),
        Normalize(mean=mean, std=std)
    ])

    return preprocessing(img.copy()).unsqueeze(0)

def show_cam_on_image(img: np.ndarray,
                      mask: np.ndarray,
                      use_rgb: bool = False,
                      colormap: int = cv2.COLORMAP_JET) -> np.ndarray:
    """ This function overlays the cam mask on the image as an heatmap.
    By default the heatmap is in BGR format.

    :param img: The base image in RGB or BGR format. 背景图像
    :param mask: The cam mask. cam掩码图像
    :param use_rgb: Whether to use an RGB or BGR heatmap, this should be set to True if 'img' is in RGB format.  是否使用rgb色彩图
    :param colormap: The OpenCV colormap to be used.
    :returns: The default image with the cam overlay.
    """
    heatmap = cv2.applyColorMap(np.uint8(255 * mask), colormap)
    if use_rgb:
        heatmap = cv2.cvtColor(heatmap, cv2.COLOR_BGR2RGB)        #颜色空间转换，BGR-RGB
    heatmap = np.float32(heatmap) / 255

    if np.max(img) > 1:
        raise Exception("The input image should np.float32 in the range [0, 1]")

    cam = heatmap + img
    cam = cam / np.max(cam)
    return np.uint8(255 * cam)    


In [2]:
def show_cam_on_image(img: np.ndarray,
                      mask: np.ndarray,
                      use_rgb: bool = False,
                      colormap: int = cv2.COLORMAP_JET) -> np.ndarray:
    """ This function overlays the cam mask on the image as an heatmap.
    By default the heatmap is in BGR format.

    :param img: The base image in RGB or BGR format. 背景图像
    :param mask: The cam mask. cam掩码图像
    :param use_rgb: Whether to use an RGB or BGR heatmap, this should be set to True if 'img' is in RGB format.  是否使用rgb色彩图
    :param colormap: The OpenCV colormap to be used.
    :returns: The default image with the cam overlay.
    """
    heatmap = cv2.applyColorMap(np.uint8(255 * mask), colormap)
    if use_rgb:
        heatmap = cv2.cvtColor(heatmap, cv2.COLOR_BGR2RGB)        #颜色空间转换，BGR-RGB
    heatmap = np.float32(heatmap) / 255

    if np.max(img) > 1:
        raise Exception("The input image should np.float32 in the range [0, 1]")

    cam = heatmap + img
    cam = cam / np.max(cam)
    return np.uint8(255 * cam)

In [3]:
grayscale_cam = cv2.imread('h1.png')
grayscale_cam = cv2.resize(grayscale_cam, (224, 224))
grayscale_cam = np.float32(grayscale_cam) / 255

pic_path = 'dataset\light_ 1350.jpg'
rgb_img = cv2.imread(pic_path)
rgb_img = cv2.resize(rgb_img, (224, 224))
rgb_img = np.float32(rgb_img) / 255
# rgb_img = preprocess_image(rgb_img, mean=[0.5, 0.5, 0.5], 
#                                     std=[0.5, 0.5, 0.5])

# heatmap = cv2.applyColorMap(np.uint8(255 * grayscale_cam), colormap = cv2.COLORMAP_JET)
# heatmap = np.float32(heatmap) / 255


cam_image = show_cam_on_image(rgb_img, grayscale_cam)
# cv2.imshow('1',cam_image)
# cv2.waitKey(0)
cv2.imwrite('pic2_cam.jpg', cam_image)

True